In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import warnings
warnings.simplefilter('ignore', DeprecationWarning)

In [2]:
data_dir = '../../../../data/'

In [3]:
#data = pd.read_csv('https://dl.dropboxusercontent.com/u/5743203/data/titanic/titanic_train.csv')
data = pd.read_csv(data_dir + 'clean_data.csv')  # result column, 1 means home win, 2 home draw, 3 home lose
data2 = pd.read_csv(data_dir + 'clean_data_using_getdummy.csv')  # result column, 1 means home win, 2 home draw, 3 home lose

In [4]:
result_column = data['result']
result_column2 = data2['result']

In [5]:
print(np.mean(result_column == 1))
print(np.mean(result_column == 2))
print(np.mean(result_column == 3))

0.468425071961
0.259531222464
0.272043705575


In [6]:
target = result_column.values
target2 = result_column2.values

In [8]:
# data.columns[0:]
numerical_features = data[data.columns[0:-1]]
numerical_features2 = data2[data2.columns[0:-2]]
numerical_features2.head()

,HomeTeam_AZ Alkmaar,HomeTeam_Aachen,HomeTeam_Ajaccio,HomeTeam_Ajax,HomeTeam_Ajax,HomeTeam_Alaves,HomeTeam_Almeria,HomeTeam_Arles,HomeTeam_Arsenal,HomeTeam_Ascoli,...,IWA,LBH,LBD,LBA,WHH,WHD,WHA,VCH,VCD,VCA
0,0,0,0,0,0,0,0,0,0,0,...,7.5,1.33,4.00,8.0,1.33,4.00,8.00,1.35,4.00,8.00
1,0,0,0,0,0,0,0,0,0,0,...,2.3,3.00,3.20,2.1,2.87,3.25,2.15,2.70,3.25,2.25
2,0,0,0,0,0,0,0,0,0,0,...,3.5,2.00,3.20,3.2,1.95,3.25,3.30,1.85,3.30,3.60
3,0,0,0,0,0,0,0,0,0,0,...,4.8,1.53,3.40,5.5,1.57,3.50,5.00,1.55,3.50,5.00
4,0,0,0,0,0,0,0,0,0,0,...,2.5,2.50,3.25,2.4,2.40,3.25,2.50,2.40,3.20,2.50


In [9]:
features_array = numerical_features.values
features_array2 = numerical_features2.values

In [17]:
from sklearn.cross_validation import train_test_split

features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=0)
features_train2, features_test2, target_train2, target_test2 = train_test_split(
    features_array2, target2, test_size=0.20, random_state=0)
print(len(features_train))
print(len(features_train2))
print(len(features_test2))
print(features_array2[0, 377])

13618
13618
3405
1.3


In [24]:
def is_unexpected(features, target, threshold):
    b365_odds = features[:, 16 : 19]
    return (((np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold) 
            & (np.argmin(b365_odds, axis = 1) + 1 != target))

def correct_predict_based_on_odds(features, target, threshold):
    b365_odds = features[:, 16:19]
    new_features = features[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    new_target = target[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    b365_odds = b365_odds[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    miss_predicts_rate = np.mean(np.argmin(b365_odds, axis = 1) + 1 != new_target)

    return new_features, new_target, miss_predicts_rate

def miss_predict_based_on_odds(features, target):
    b365_odds = features[:, 16 : 19]
    return np.argmin(b365_odds, axis = 1) + 1 != target    

def is_unexpected2(features, target, threshold):
    b365_odds = features[:, 377 : 380]  # This needs to be changed based on different result.csv
    return (((np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold) 
            & (np.argmin(b365_odds, axis = 1) + 1 != target))

def correct_predict_based_on_odds2(features, target, threshold):
    b365_odds = features[:, 377:380]  # This needs to be changed based on different result.csv
    new_features = features[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    new_target = target[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    b365_odds = b365_odds[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    miss_predicts_rate = np.mean(np.argmin(b365_odds, axis = 1) + 1 != new_target)

    return new_features, new_target, miss_predicts_rate

def miss_predict_based_on_odds2(features, target):
    b365_odds = features[:, 377 : 380] # This needs to be changed based on different result.csv
    return np.argmin(b365_odds, axis = 1) + 1 != target  

def correct_predict_based_on_odds_with_index(features, target, threshold, start, end):
    b365_odds = features[:, start:end]  # This needs to be changed based on different result.csv
    new_features = features[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    new_target = target[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    b365_odds = b365_odds[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    miss_predicts_rate = np.mean(np.argmin(b365_odds, axis = 1) + 1 != new_target)

    return new_features, new_target, miss_predicts_rate

def miss_predict_based_on_odds_with_index(features, target, start, end):
    b365_odds = features[:, start : end] # This needs to be changed based on different result.csv
    return np.argmin(b365_odds, axis = 1) + 1 != target  

for threshold in np.arange(0.5, 5, 0.5):
    new_features2, new_target2, miss_predicts_rate2 = correct_predict_based_on_odds2(features_train2, target_train2, threshold)
    print(len(new_features2), miss_predicts_rate2)
    
print(np.mean(miss_predict_based_on_odds2(features_train2, target_train2)))

13109 0.462659241742
10595 0.43378952336
8337 0.396665467194
6946 0.375611862943
5893 0.354318683183
5048 0.334587955626
4236 0.313503305005
3553 0.29271038559
2987 0.268496819551
0.468864737847


In [25]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(features_train)  
X_train = scaler.transform(features_train)  
# apply same transformation to test data
X_test = scaler.transform(features_test)  
# print(X_train[0:5])

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler  


In [22]:
clf = LogisticRegression(C=1, n_jobs = -1)
# clf.fit(X_train, target_train)

# from sklearn.svm import SVC
# clf = SVC(C = 1, kernel = 'rbf', probability = True)
# clf.fit(X_train, target_train)

# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(min_samples_split = 10, n_estimators = 1000, criterion='entropy')
# clf.fit(X_train, target_train)

# from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier(n_estimators = 1000)
# clf.fit(X_train, target_train)

# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()
# clf.fit(X_train, target_train)

for threshold in np.arange(0.1, 7, 0.5):
    new_features_train, new_target_train, miss_predicts_rate = correct_predict_based_on_odds(features_train, target_train, threshold)
    new_features_test, new_target_test, miss_predicts_rate_test = correct_predict_based_on_odds(features_test, target_test, threshold)
    print('Odds Difference Threshold to remove sample data: ', threshold)
    print('Number of training data under this odds difference threshold: ', len(new_features_train))
    print('Number of test data under this odds difference threshold: ', len(new_features_test))
    
    # feature scaling
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(new_features_train)  
    X_train = scaler.transform(new_features_train)  
    # apply same transformation to test data
    X_test = scaler.transform(new_features_test)  
    
    
    clf.fit(X_train, new_target_train)
    target_predicted_proba = clf.predict_proba(X_test)
    # print(target_predicted_proba)
    target_predicted = clf.predict(X_test)
    prob_threshold = 0.75
    
    print('Percentage of predictions with the Prob_threshold of ', prob_threshold, 
          ' : ', np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
    print('Accuracy with this prob_threshold: ', np.mean((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)) 
          / np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
    #print(np.mean((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    print('Total accuracy: ', accuracy_score(new_target_test, target_predicted))
    X_test_greater_than_prob_threshold = features_train[np.amax(target_predicted_proba, axis = 1) > prob_threshold, ]
#     print(X_test_greater_than_prob_threshold[0 : 2, ])
    print('\n\n')

Odds Difference Threshold to remove sample data:  0.1
Number of training data under this odds difference threshold:  13616
Number of test data under this odds difference threshold:  3405
Percentage of predictions with the Prob_threshold of  0.75  :  0.0825256975037
Accuracy with this prob_threshold:  0.779359430605
Total accuracy:  0.51718061674



Odds Difference Threshold to remove sample data:  0.6
Number of training data under this odds difference threshold:  13078
Number of test data under this odds difference threshold:  3252
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 3405
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 3252


 0.75  :  0.0870233702337
Accuracy with this prob_threshold:  0.780918727915
Total accuracy:  0.524292742927



Odds Difference Threshold to remove sample data:  1.1
Number of training data under this odds difference threshold:  10007
Number of test data under this odds difference threshold:  2477
Percentage of predictions with the Prob_threshold of  0.75  :  0.111021396851
Accuracy with this prob_threshold:  0.778181818182
Total accuracy:  0.562373839322



Odds Difference Threshold to remove sample data:  1.6
Number of training data under this odds difference threshold:  8057
Number of test data under this odds difference threshold:  2020
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 2477
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 2020


 0.75  :  0.134653465347
Accuracy with this prob_threshold:  0.772058823529
Total accuracy:  0.589108910891



Odds Difference Threshold to remove sample data:  2.1
Number of training data under this odds difference threshold:  6609
Number of test data under this odds difference threshold:  1622
Percentage of predictions with the Prob_threshold of  0.75  :  0.167694204686
Accuracy with this prob_threshold:  0.772058823529
Total accuracy:  0.618988902589



Odds Difference Threshold to remove sample data:  2.6
Number of training data under this odds difference threshold:  5752
Number of test data under this odds difference threshold:  1400
Percentage of predictions with the Prob_threshold of 

...

 0.75  :  0.383125864454
Accuracy with this prob_threshold:  0.750902527076
Total accuracy:  0.709543568465



Odds Difference Threshold to remove sample data:  5.1
Number of training data under this odds difference threshold:  2633
Number of test data under this odds difference threshold:  637
Percentage of predictions with the Prob_threshold of  0.75  :  0.43328100471
Accuracy with this prob_threshold:  0.735507246377
Total accuracy:  0.726844583987



Odds Difference Threshold to remove sample data:  5.6
Number of training data under this odds difference threshold:  2264
Number of test data under this odds difference threshold:  550
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 637
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 550


 0.75  :  0.505454545455
Accuracy with this prob_threshold:  0.744604316547
Total accuracy:  0.743636363636



Odds Difference Threshold to remove sample data:  6.1
Number of training data under this odds difference threshold:  1923
Number of test data under this odds difference threshold:  478
Percentage of predictions with the Prob_threshold of  0.75  :  0.543933054393
Accuracy with this prob_threshold:  0.753846153846
Total accuracy:  0.751046025105



Odds Difference Threshold to remove sample data:  6.6
Number of training data under this odds difference threshold:  1720
Number of test data under this odds difference threshold:  419
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 478
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:50: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 419


 0.75  :  0.613365155131
Accuracy with this prob_threshold:  0.750972762646
Total accuracy:  0.749403341289





In [27]:
clf = LogisticRegression(C=1, n_jobs = -1)


# from sklearn.svm import SVC
# clf = SVC(C = 1, kernel = 'poly', probability = True)


# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(min_samples_split = 10, n_estimators = 1000, criterion='entropy')


# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.tree import DecisionTreeClassifier
# clf = AdaBoostClassifier()


# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()


for threshold in np.arange(0.1, 7, 0.5):
    new_features_train2, new_target_train2, miss_predicts_rate2 = correct_predict_based_on_odds2(features_train2, target_train2, threshold)
    new_features_test2, new_target_test2, miss_predicts_rate_test2 = correct_predict_based_on_odds2(features_test2, target_test2, threshold)
    print('Odds Difference Threshold to remove sample data: ', threshold)
    print('Number of training data under this odds difference threshold: ', len(new_features_train2))
    print('Number of test data under this odds difference threshold: ', len(new_features_test2))
    
    # feature scaling
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(new_features_train2)  
    X_train2 = scaler.transform(new_features_train2)  
    # apply same transformation to test data
    X_test2 = scaler.transform(new_features_test2)  
    
    
    clf.fit(X_train2, new_target_train2)
    target_predicted_proba2 = clf.predict_proba(X_test2)
    # print(target_predicted_proba)
    target_predicted2 = clf.predict(X_test2)
    prob_threshold = 0.70
    
    print('Percentage of predictions with the Prob_threshold of ', prob_threshold, 
          ' : ', np.mean(np.amax(target_predicted_proba2, axis = 1) > prob_threshold))
    print('Accuracy with this prob_threshold: ', np.mean((target_predicted2 == new_target_test2) & (np.amax(target_predicted_proba2, axis = 1) > prob_threshold)) 
          / np.mean(np.amax(target_predicted_proba2, axis = 1) > prob_threshold))   
    #print(np.mean((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    print('Total accuracy: ', accuracy_score(new_target_test2, target_predicted2))
    X_test_greater_than_prob_threshold2 = features_train[np.amax(target_predicted_proba2, axis = 1) > prob_threshold, ]
#     print(X_test_greater_than_prob_threshold[0 : 2, ])
    print("precision score: ", precision_score( new_target_test2, target_predicted2, average = None ))
    print('\n\n')

Odds Difference Threshold to remove sample data:  0.1
Number of training data under this odds difference threshold:  13616
Number of test data under this odds difference threshold:  3405
Percentage of predictions with the Prob_threshold of  0.7  :  0.137738619677
Accuracy with this prob_threshold:  0.710021321962
Total accuracy:  0.502790014684
precision score:  [ 0.52936206  0.30241935  0.48607595]



Odds Difference Threshold to remove sample data:  0.6
Number of training data under this odds difference threshold:  13078
Number of test data under this odds difference threshold:  3252
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 3405
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 3252


 0.7  :  0.146371463715
Accuracy with this prob_threshold:  0.710084033613
Total accuracy:  0.510455104551
precision score:  [ 0.53798858  0.28695652  0.49530201]



Odds Difference Threshold to remove sample data:  1.1
Number of training data under this odds difference threshold:  10007
Number of test data under this odds difference threshold:  2477
Percentage of predictions with the Prob_threshold of  0.7  :  0.204279370206
Accuracy with this prob_threshold:  0.697628458498
Total accuracy:  0.541380702463
precision score:  [ 0.56958474  0.3         0.52815534]



Odds Difference Threshold to remove sample data:  1.6
Number of training data under this odds difference threshold:  8057
Number of test data under this odds difference threshold:  2020
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 2477
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 2020


 0.7  :  0.256930693069
Accuracy with this prob_threshold:  0.687861271676
Total accuracy:  0.565841584158
precision score:  [ 0.5982562   0.25185185  0.55076142]



Odds Difference Threshold to remove sample data:  2.1
Number of training data under this odds difference threshold:  6609
Number of test data under this odds difference threshold:  1622
Percentage of predictions with the Prob_threshold of  0.7  :  0.339704069051
Accuracy with this prob_threshold:  0.687840290381
Total accuracy:  0.607274969174
precision score:  [ 0.63358147  0.32727273  0.6039604 ]



Odds Difference Threshold to remove sample data:  2.6
Number of training data under this odds difference threshold:  5752
Number of test data under this odds difference threshold:  1400
Percentage of predictions with the Prob_threshold of 

...

 0.7  :  0.643153526971
Accuracy with this prob_threshold:  0.696774193548
Total accuracy:  0.656984785615
precision score:  [ 0.7025641   0.27777778  0.52941176]



Odds Difference Threshold to remove sample data:  5.1
Number of training data under this odds difference threshold:  2633
Number of test data under this odds difference threshold:  637
Percentage of predictions with the Prob_threshold of  0.7  :  0.675039246468
Accuracy with this prob_threshold:  0.7
Total accuracy:  0.671899529042
precision score:  [ 0.71346154  0.23333333  0.57471264]



Odds Difference Threshold to remove sample data:  5.6
Number of training data under this odds difference threshold:  2264
Number of test data under this odds difference threshold:  550
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 637
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 550


 0.7  :  0.701818181818
Accuracy with this prob_threshold:  0.704663212435
Total accuracy:  0.690909090909
precision score:  [ 0.73478261  0.21212121  0.61403509]



Odds Difference Threshold to remove sample data:  6.1
Number of training data under this odds difference threshold:  1923
Number of test data under this odds difference threshold:  478
Percentage of predictions with the Prob_threshold of  0.7  :  0.719665271967
Accuracy with this prob_threshold:  0.706395348837
Total accuracy:  0.68410041841
precision score:  [ 0.73267327  0.2         0.61538462]



Odds Difference Threshold to remove sample data:  6.6
Number of training data under this odds difference threshold:  1720
Number of test data under this odds difference threshold:  419
Percentage of predictions with the Prob_threshold of 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 478
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 13618 but corresponding boolean dimension is 419


 0.7  :  0.737470167064
Accuracy with this prob_threshold:  0.708737864078
Total accuracy:  0.682577565632
precision score:  [ 0.73714286  0.22580645  0.55263158]





In [77]:
## Now let's train the games only labeld as upset or non-upset games
games_upset = pd.read_csv(data_dir + 'result-teamname2.csv')
games_upset.drop(['FTHG', 'FTAG', 'HTHG', 'HTAG'], axis = 1, inplace=True)
games_upset.columns[377:380]

Index(['B365H', 'B365D', 'B365A'], dtype='object')

In [78]:
result_column = games_upset['upsetx']
np.mean(result_column == 1), np.mean(result_column == 0)

(0.30623274393467659, 0.69376725606532341)

In [79]:
# result_column[result_column == 1.0] = 2
# result_column[result_column == 0.0] = 1
# np.mean(result_column == 1), np.mean(result_column == 2)


In [81]:
# np.mean(result_column == 1), np.mean(result_column == 2)

In [82]:
target = result_column.values

In [83]:
numerical_features = games_upset[games_upset.columns[0:-1]]
features_array = numerical_features.values

features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=0)

In [88]:


# clf = LogisticRegression(C=1000, n_jobs = -1)


# from sklearn.svm import SVC
# clf = SVC(C = 1,kernel = 'rbf',class_weight = "balanced")


# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
clf = AdaBoostClassifier()


# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()

# feature scaling
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(features_train)  
X_train = scaler.transform(features_train)  
# apply same transformation to test data
X_test = scaler.transform(features_test)  


clf.fit(X_train, target_train)
# target_predicted_proba = clf.predict_proba(X_test)
# print(target_predicted_proba)
target_predicted = clf.predict(X_test)
# prob_threshold = 0.70

# print('Percentage of predictions with the Prob_threshold of ', prob_threshold, 
#       ' : ', np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
# print('Accuracy with this prob_threshold: ', np.mean((target_predicted == target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)) 
#       / np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
#print(np.mean((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
#print(np.sum((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
#print(np.sum((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))

In [89]:

print('Total accuracy: ', accuracy_score(target_test, target_predicted))
# X_test_greater_than_prob_threshold = features_train[np.amax(target_predicted_proba, axis = 1) > prob_threshold, ]
#     print(X_test_greater_than_prob_threshold[0 : 2, ])
print("precision score: ", precision_score( target_test, target_predicted ))
print('\n\n')

target_predicted[200:220], target_test[200:220]

Total accuracy:  0.696035242291
precision score:  0.509072580645





(array([ 0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,
         0.,  0.,  1.,  1.,  0.,  0.,  0.]),
 array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  1.,  0.]))

In [90]:
start = 377
end = 380
for threshold in np.arange(0.1, 7, 0.5):
    new_features_train, new_target_train, miss_predicts_rate = correct_predict_based_on_odds_with_index(features_train, target_train, threshold, start, end)
    new_features_test, new_target_test, miss_predicts_rate_test = correct_predict_based_on_odds_with_index(features_test, target_test, threshold, start, end)
    print('Odds Difference Threshold to remove sample data: ', threshold)
    print('Number of training data under this odds difference threshold: ', len(new_features_train))
    print('Number of test data under this odds difference threshold: ', len(new_features_test))
    
    # feature scaling
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(new_features_train)  
    X_train = scaler.transform(new_features_train)  
    # apply same transformation to test data
    X_test = scaler.transform(new_features_test)  
    
    
    clf.fit(X_train, new_target_train)
    target_predicted_proba = clf.predict_proba(X_test)
    # print(target_predicted_proba)
    target_predicted = clf.predict(X_test)
    prob_threshold = 0.50
    
    print('Percentage of predictions with the Prob_threshold of ', prob_threshold, 
          ' : ', np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
    print('Accuracy with this prob_threshold: ', 
          np.mean((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)) 
          / np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
    #print(np.mean((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    print('Total accuracy: ', accuracy_score(new_target_test, target_predicted))
#     X_test_greater_than_prob_threshold = features_train[np.amax(target_predicted_proba, axis = 1) > prob_threshold, ]
#     print(X_test_greater_than_prob_threshold[0 : 2, ])
    print("precision score total: ", precision_score( new_target_test, target_predicted, average = None ))
    true_positive = np.sum((target_predicted == new_target_test) 
                           & (np.amax(target_predicted_proba, axis = 1) > prob_threshold) 
                           & (target_predicted == 1))
    false_positive = np.sum((target_predicted != new_target_test) 
                           & (np.amax(target_predicted_proba, axis = 1) > prob_threshold) 
                           & (target_predicted == 1))    
    print("precision score under this prob_threshold: ", float(true_positive) / (float(true_positive + false_positive))) 
    print('\n\n')

Odds Difference Threshold to remove sample data:  0.1
Number of training data under this odds difference threshold:  13616
Number of test data under this odds difference threshold:  3405
Percentage of predictions with the Prob_threshold of  0.5  :  1.0
Accuracy with this prob_threshold:  0.696328928047
Total accuracy:  0.696328928047
precision score total:  [ 0.77299089  0.50958628]
precision score under this prob_threshold:  0.5095862764883956



Odds Difference Threshold to remove sample data:  0.6
Number of training data under this odds difference threshold:  13078
Number of test data under this odds difference threshold:  3252
Percentage of predictions with the Prob_threshold of  0.5  :  1.0
Accuracy with this prob_threshold:  0.689421894219
Total accuracy:  0.689421894219
precision score total:  [ 0.72998066  0.53223388]
precision score under this prob_threshold:  0.5322338830584707



Odds Difference Threshold to remove sample data:  1.1
Number of training data under this odds di